In [12]:
import os
from os import listdir
from os.path import isfile, join

import json
from urllib.request import urlopen, Request
from lxml import html
import requests
from bs4 import BeautifulSoup
import re

import pickle
import time

import sys
sys.path.append('src/')
import preprocessing
# import utils

%load_ext autoreload
%autoreload 2

def load_data(filepath):
    with open(filepath, 'r') as f:
        return json.load(f)

def save_data(filepath, data):
    with open(filepath, 'w') as f:
        json.dump(data, f, indent=4)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [82]:

# find the index of https://philpapers.org/rec/SMUQAF
last_idx = url_list.index("https://philpapers.org/rec/SMUQAF")

url_list = url_list[last_idx:]


In [85]:



def extract_categories(soup):
    # Finding the <div> with the specific id and itemprop
    categories_div = soup.find('div', {'id': 'record-categories', 'itemprop': 'keywords'})
    if not categories_div:
        return "No categories found"
    
    # Extracting category details from each <div> within the main <div>
    categories = []
    for div in categories_div.find_all('div'):
        cat_name = div.find('a', class_='catName').text if div.find('a', class_='catName') else None
        cat_area = div.find('a', class_='catArea').text if div.find('a', class_='catArea') else None
        if cat_name and cat_area:
            categories.append({'Category Name': cat_name, 'Category Area': cat_area})
    return categories

def extract_keywords(soup):
    # First, find all <div> elements with itemprop="keywords"
    all_keywords_divs = soup.find_all('div', class_="")

    # We will filter these to exclude those that are part of the categories section
    for div in all_keywords_divs:
        # Check if this div is a direct child of the categories section (by checking parent properties)
        if div.parent.get('itemprop') == 'keywords':
            continue  # Skip this div, it's part of categories

        # Assuming the correct keywords div has a class value that is empty
        if div.find('div', itemprop='keywords'):
            keywords = [keyword.text.strip() for keyword in div.find_all('a')]
            return keywords

    return "No appropriate keywords section found"

def extract_meta(soup):
    # title: <h1 class="recTitle" itemprop="name"
    title = soup.find('h1', class_='recTitle').text
    # auhor: <div class="recAuthors"
    author = soup.find('div', class_='recAuthors').text
    # pubinfo: <div class="recPubInfo"
    pubinfo = soup.find('div', class_='recPubInfo').text
    return {
        "title": title,
        "author": author,
        "pubinfo": pubinfo
    }

def get_categories_and_keywords(url, headers_list):
    headers_index = 0
    headers = headers_list[headers_index]
    response = requests.get(url, headers=headers)
    while response.status_code == 429:
        retry_after = int(response.headers.get("Retry-After", 35))
        print(f"Rate limit hit. Sleeping for {retry_after} seconds.")
        time.sleep(retry_after)
        headers_index = (headers_index + 1) % len(headers_list)  # Rotate headers
        headers = headers_list[headers_index]
        response = requests.get(url, headers=headers)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, "html.parser")

    # extracting categories and keywords
    categories = extract_categories(soup)
    keywords = extract_keywords(soup)

    # extracting metadata
    meta = extract_meta(soup)

    # print("Categories:", categories)
    # print("Keywords:", keywords)
    item_dict = {
        "url": url,
        "meta": meta,
        "categories": categories,
        "keywords": keywords,
        "all_info": soup.prettify()
    }
    return item_dict

def process_urls():
    enhanced_biblio_url_list = url_list
    # problem_list = {}

    headers_list = [
        {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'},
        {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:125.0) Gecko/20100101 Firefox/125.0'},
        {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}
    ]

    for url in enhanced_biblio_url_list:
        try:
            item_dict = get_categories_and_keywords(url, headers_list)
            file_name = url.split('/')[-1]
            save_data("taxonomy_data/" + file_name + ".json", item_dict)
            print("LOADED: ", url)
        except Exception as e:
            print("ERROR: ", url, str(e))
            # problem_list[
            # save
            with open("taxonomy_data/to_check/problem_list.json", 'a') as f:
                json.dump({url: str(e)}, f, indent=4)
            time.sleep(5)  # Simple retry mechanism

# Main execution
if __name__ == "__main__":
    process_urls()


ERROR:  https://philpapers.org/rec/LACGKA 'NoneType' object has no attribute 'text'


Attatch keywords to entries

In [4]:
# open all the json files in the directory taxonomy_data/
import os
import json




def get_all_info(directory):
    data = {}
    for filename in tqdm(os.listdir(directory)):
        if filename.endswith(".json"):
            info_dict = load_data(os.path.join(directory, filename))
            extracted_info_dict = {
                # "url": info_dict["url"],
                # "meta": info_dict["meta"],
                "categories": info_dict["categories"],
                "keywords": info_dict["keywords"]
            }
            data[info_dict["url"]] = extracted_info_dict
            # tqdm.write(f"Loaded {len(data)} files")

    # save to database/sep_clean
    save_data("taxonomy_data/sep_clean/philpaper_keywords.json", data)
    return data

item_dict_list = get_all_info("taxonomy_data/")
print(len(item_dict_list))

  0%|          | 0/69784 [00:00<?, ?it/s]

69781


TypeError: unhashable type: 'slice'

In [16]:
# info dict of entry and bibliography
# load "taxonomy_data/sep_clean/enhanced_bibliographies.json"
ent_bibli_dict = preprocessing.load_data("database/sep_clean/enhanced_bibliographies.json")

# load "taxonomy_data/sep_clean/philpaper_keywords.json"
philpaper_keywords_dict = preprocessing.load_data("database/sep_clean/philpaper_keywords.json")

# merge the philpaper_keywords_dict into the items of ent_bibli_dict[url]["philpaper_archived_url_list"]
for url, item in tqdm(ent_bibli_dict.items()):
    item["philpaper_archived"] = {}
    for philpaper_url in item["philpaper_archived_url_list"]:
        if philpaper_url in philpaper_keywords_dict:
            # only take "Category Name" from categories and "keywords" from keywords
            info_dict = philpaper_keywords_dict[philpaper_url]
            # categories = [] if "No categories found"
            # keywords = [] if "No appropriate keywords section found"
            cat = [i["Category Name"] for i in info_dict["categories"]] if info_dict["categories"] != "No categories found" else []
            kw = info_dict["keywords"] if info_dict["keywords"] != "No appropriate keywords section found" else []

            extracted_info_dict = {
                "categories": cat,
                "keywords": kw
            }
            item["philpaper_archived"][philpaper_url] = extracted_info_dict
    # remove philpaper_archived_url_list
    item.pop("philpaper_archived_url_list")
    # print(item)
    # break

# save to database/sep_clean
preprocessing.save_data("database/sep_clean/entry_bibli_categories_keywords.json", ent_bibli_dict)

  0%|          | 0/1796 [00:00<?, ?it/s]

## Taxonomy



determine path_all_parents by 

first, locate the level of the current entry

for example, "forgiveness" is 5: "Subtopics"

so we start from level 5

then gradually reduce the level as treversing through its parents

so we get something like:

{"level1":, # all parents (clusters) of level 2
"level2": # all parents of level 3 parents
"level3":((329,"Mental States and Processes",),(30,"Normative Ethics"),(6143,"Practical Reason")), # these are all parents of 29,
"level4": (29, "Moral Psychology"), # this is the parent of 4712
"level5": (4712,"Moral States and Processes")

if the entry start from level 4, just leave level 5 as blank 

In [21]:
import json

def read_json_file(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

def write_json_file(data, file_path):
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)

def build_parent_map(categories):
    parent_map = {1: {"name": "Philosophy", "parent_ids": [], "primary_parent_id": None}}  # Include root category
    for category in categories:
        parent_map[category[1]] = {
            "name": category[0],
            "parent_ids": list(map(int, category[2].split(','))),
            "primary_parent_id": category[3]
        }
    return parent_map

def determine_category_level(current_id, parent_map):
    level = 1
    while current_id != 1:
        if parent_map[current_id]["primary_parent_id"] == 1:
            break
        current_id = parent_map[current_id]["primary_parent_id"]
        level += 1
    return level

def level_to_name(level):
    return {
        0: "Root",
        1: "Clusters",
        2: "Areas",
        3: "Subareas",
        4: "Topics",
        5: "Subtopics"
    }.get(level, "Unknown")

def update_categories_with_full_paths(categories):
    parent_map = build_parent_map(categories)
    updated_categories = {
        "1": {
        "title": "Philosophy",
        "level": "0 Root",
        "primary_parent": [],
        "path_primary_parents": [],
        "path_all_parents": []}
    }
    
    for category in categories:
        current_id = category[1]
        primary_path = []
        all_parents_path = []

        level = determine_category_level(current_id, parent_map)
        
        # Traverse through primary parents to root
        while current_id != 1:
            category_info = parent_map.get(current_id, None)
            if not category_info:
                break
            if category_info['primary_parent_id'] != 1:
                # primary_path.append(str((category_info['primary_parent_id'], parent_map[category_info['primary_parent_id']]['name'])))
                primary_path.append((category_info['primary_parent_id']))
            current_id = category_info['primary_parent_id']

        # traverse through all parents to root
        current_id = category[1]
        for lvl in range(1, level+1):
            # all_parents_path.append([str((id, parent_map[id]['name'])) for id in map(int, parent_map[current_id]['parent_ids']) if id in parent_map])
            all_parents_path.append([id for id in map(int, parent_map[current_id]['parent_ids']) if id in parent_map])
            current_id = parent_map[current_id]['primary_parent_id']

        # Reverse to maintain order from root to the category
        primary_path.reverse()
        all_parents_path.reverse()

        # Create the detailed path for all parents
        # for lvl in range(1, level+1):
        #     all_parents_path[f"level{lvl}"] = [(id, parent_map[id]['name']) for id in map(int, categories[lvl-1][2].split(',')) if id in parent_map]

        # get all parents found during traversal
        # put them in the correct level

        # Prepare the updated category entry as a dictionary
        updated_categories[category[1]] = {
            "title": category[0],
            "level": f"{level} {level_to_name(level)}",
            "primary_parent": str((category[3], parent_map[category[3]]['name'])),
            "path_primary_parents": primary_path,
            "path_all_parents": all_parents_path
        }
    
    return updated_categories

def main():
    input_file = 'PhilPaper/philpaper_taxonomy.json'
    output_file = 'PhilPaper/philpaper_taxonomy_full_path.json'
    
    categories = read_json_file(input_file)
    updated_categories = update_categories_with_full_paths(categories)
    write_json_file(updated_categories, output_file)
    print(f'Updated categories have been written to {output_file}')

if __name__ == "__main__":
    main()


Updated categories have been written to PhilPaper/philpaper_taxonomy_full_path.json


### correspond taxonomy to entries


In [32]:
# load stanford_encyclopedia/PhilPaper/philpaper_taxonomy_full_path.json
philpaper_taxonomy_full_path = load_data("PhilPaper/philpaper_taxonomy_full_path.json")

# load database/sep_clean/entry_bibli_categories_keywords.json
entry_bibli_categories_keywords = load_data("database/sep_clean/entry_bibli_categories_keywords.json")

# for each philpaper_archived work in each entry
# find "title" in dict in philpaper_taxonomy_full_path that matches "categories" in dict in entry_bibli_categories_keywords
# add "full_path" to entry_bibli_categories_keywords


# we need 
# 1/ label (identify categories) of cited archived works from philpaper assigned categories and keywords
# 2/ label categories of the entry based on its cited works' labels


for url, item in tqdm(entry_bibli_categories_keywords.items()): # for each entry
    count_dict = {
        62: 0
        }
    for philpaper_url, philpaper_item in item["philpaper_archived"].items(): # for each philpaper archived work cited in the entry
        philpaper_item["categories_with_path"] = {}
        for cat in philpaper_item["categories"]:
            # start search for path of this cat
            for cat_id, cat_item in philpaper_taxonomy_full_path.items():
                if cat == cat_item["title"]:
                    path_primary_parents = cat_item["path_primary_parents"]
                    path_all_parents = cat_item["path_all_parents"]
                    # philpaper_item["categories_with_path"][cat_id] = {
                    #     "path_primary_parents": cat_item["path_primary_parents"],
                    #     "path_all_parents": cat_item["path_all_parents"]
                    #     }
                    break # end search
            # identify categories
            for k,v in count_dict.items():
                if k in path_primary_parents: # only consider primary parents for now
                    count_dict[k] += 1
                    break
    if count_dict[62] > 0:
        print(url, count_dict[62])
    # print(count_dict)
             

# save_data("database/sep_clean/entry_bibli_cat_with_path.json", entry_bibli_categories_keywords)


  0%|          | 0/1796 [00:00<?, ?it/s]

https://philpapers.org/sep/music/ 2
https://philpapers.org/sep/wang-yangming/ 18
https://philpapers.org/sep/nishida-kitaro/ 3
https://philpapers.org/sep/kyoto-school/ 4
https://philpapers.org/sep/school-names/ 37
https://philpapers.org/sep/ibn-arabi/ 1
https://philpapers.org/sep/ethics-ai/ 1
https://philpapers.org/sep/mysticism/ 1
https://philpapers.org/sep/daoism-religion/ 14
https://philpapers.org/sep/chinese-social-political/ 19
https://philpapers.org/sep/determinate-determinables/ 1
https://philpapers.org/sep/kant-aesthetics/ 2
https://philpapers.org/sep/chinese-metaphysics/ 56
http://philpapers.org/sep/aesthetics-of-everyday/ 2
https://philpapers.org/sep/knowledge-how/ 4
https://philpapers.org/sep/chinese-epistemology/ 37
http://philpapers.org/sep/contradiction/ 1
https://philpapers.org/sep/confucian-gender/ 21
https://philpapers.org/sep/confucianism-modern/ 13
https://philpapers.org/sep/chinese-logic-language/ 45
https://philpapers.org/sep/comparphil-chiwes/ 49
https://philpapers